In [1]:
import pandas as pd
import numpy as np
import pickle

In [2]:
df = pickle.load(open('drugbank_embds.pkl', 'rb'))
df = df[~pd.isna(df['target_embd'])]
df.head()

,drug_cas,drug_id,drug_name,target_actions,target_amino,target_gene,target_id,target_name,target_organism,pubchem_id,pubchem_name,target_gene_fingerprint,drug_fingerprint,SMILE,target_embd
0,128270-60-0,DB00006,Bivalirudin,[inhibitor],>lcl|BSEQ0016004|Prothrombin\nMAHVRGLQLPGCLALA...,>lcl|BSEQ0016005|Prothrombin (F2)\nATGGCGCACGT...,BE0000048,Prothrombin,Human,16129704,Bivalirudin,[1.2739266e-05 -2.7748918e-05 0.00095751503 0....,[0.10044218 0.18559723 -0.12865402 -0.0650997 ...,CCC(C)C(C(=O)N1CCCC1C(=O)NC(CCC(=O)O)C(=O)NC(C...,0.0011788049 -0.00059948955 0.00048735933 -0.0...
1,65807-02-5,DB00014,Goserelin,[agonist],>lcl|BSEQ0036957|Lutropin-choriogonadotropic h...,>lcl|BSEQ0010174|Lutropin-choriogonadotropic h...,BE0000134,Lutropin-choriogonadotropic hormone receptor,Human,5311128,goserelin,[1.2739266e-05 -2.7748918e-05 0.00095751503 0....,[0.1331214 0.2391842 -0.14361311 -0.14107014 0...,CC(C)CC(C(=O)NC(CCCN=C(N)N)C(=O)N1CCCC1C(=O)NN...,-2.7273058e-05 -0.0014635038 -0.000130956 0.00...
2,65807-02-5,DB00014,Goserelin,[agonist],>lcl|BSEQ0000405|Gonadotropin-releasing hormon...,>lcl|BSEQ0018926|Gonadotropin-releasing hormon...,BE0000203,Gonadotropin-releasing hormone receptor,Human,5311128,goserelin,[1.2739266e-05 -2.7748918e-05 0.00095751503 0....,[0.1331214 0.2391842 -0.14361311 -0.14107014 0...,CC(C)CC(C(=O)NC(CCCN=C(N)N)C(=O)N1CCCC1C(=O)NN...,-0.00060385827 -0.0011927614 -0.00020535699 -0...
3,16679-58-6,DB00035,Desmopressin,[agonist],>lcl|BSEQ0000583|Vasopressin V2 receptor\nMLMA...,>lcl|BSEQ0010065|Vasopressin V2 receptor (AVPR...,BE0000293,Vasopressin V2 receptor,Human,16051933,DESMOPRESSIN,[1.2739266e-05 -2.7748918e-05 0.00095751503 0....,[0.051722348 0.24374117 -0.16236018 -0.1723347...,C1CC(N(C1)C(=O)C2CSSCCC(=O)NC(C(=O)NC(C(=O)NC(...,0.0002876672 9.454535e-06 -0.00021410194 -0.00...
4,16679-58-6,DB00035,Desmopressin,NaN,>lcl|BSEQ0009979|Vasopressin V1a receptor\nMRL...,>lcl|BSEQ0009980|Vasopressin V1a receptor (AVP...,BE0000165,Vasopressin V1a receptor,Human,16051933,DESMOPRESSIN,[1.2739266e-05 -2.7748918e-05 0.00095751503 0....,[0.051722348 0.24374117 -0.16236018 -0.1723347...,C1CC(N(C1)C(=O)C2CSSCCC(=O)NC(C(=O)NC(C(=O)NC(...,-0.00047414153 -0.00063484476 0.00041864807 -0...


In [4]:
d1 = df[['drug_id', 'drug_name', 'SMILE']].drop_duplicates()
d1.columns = ['id', 'name', 'repr']
d1['Type'] = 'drug'
d2 = df[['target_id', 'target_name', 'target_gene']].drop_duplicates()
d2.columns = ['id', 'name', 'repr']
d2['Type'] = 'target'

In [5]:
ip = pd.concat([d1, d2]).reset_index().drop(columns='index')

In [19]:
ip.to_json('searchbar.json', orient='records')

In [11]:
ip_dict = ip[['id', 'name']].set_index('id')['name'].to_dict()
pickle.dump(ip_dict, open("id_name.pkl", "wb"))

In [3]:
def getData(x):
    if type(x) == list:
        return [float(i) for i in x[0].split(' ')]
    else:
        return [float(i) for i in x.split(' ')]

In [4]:
df['target_embd'] = df['target_embd'].apply(getData)
df['drug_fingerprint'] = df['drug_fingerprint'].apply(getData)

In [5]:
d = dict()
for item in np.unique(df['drug_id'].values):
    try:
        _ = d[item]
    except:
        d.update({item : np.array(df[df['drug_id'] == item]['drug_fingerprint'].values[0]) })
t = dict()
for item in np.unique(df['target_id'].values):
    try:
        _ = t[item]
    except:
        t.update({item : np.array(df[df['target_id'] == item]['target_embd'].values[0]) })

In [6]:
pickle.dump(d, open("EmbedDict_drug.pkl", "wb"))
pickle.dump(t,open("EmbedDict_target.pkl", "wb"))

In [7]:
ndf = pd.read_csv('negative_samples/negative_samples_thrsh:12')[['node_0','node_1','shortest_path']]

In [8]:
data_1 = pd.concat([pd.DataFrame(df['target_embd'].values.tolist(), columns=['Target_Vec_'+str(i) for i in range(0, 30)]), 
               pd.DataFrame(df['drug_fingerprint'].values.tolist(), columns=['Drug_Vec_'+str(i) for i in range(0, 100)])], axis=1)
data_1['Class'] = 1

In [9]:
d = dict()
for item in np.unique(df['drug_id'].values):
    try:
        _ = d[item]
    except:
        d.update({item : df[df['drug_id'] == item]['drug_fingerprint'].values[0] })

for item in np.unique(df['target_id'].values):
    try:
        _ = d[item]
    except:
        d.update({item : df[df['target_id'] == item]['target_embd'].values[0] })

In [10]:
def getVec(n1, n2):
    try:
        if n1.startswith('DB'):
            drug = d[n1]
            target = d[n2]
        else:
            drug = d[n2]
            target = d[n1]
        return target, drug
    except:
        return np.nan, np.nan

In [11]:
ndf[['target_embd', 'drug_fingerprint']] = ndf.apply(lambda x: getVec(x.node_0, x.node_1), 
                                                         axis=1, result_type='expand')
ndf = ndf[~pd.isna(ndf['target_embd'])]
ndf['len'] = ndf['target_embd'].apply(lambda x: len(x))
ndf = ndf[ndf['len'] == 30]
ndf['len2'] = ndf['drug_fingerprint'].apply(lambda x: len(x))
ndf = ndf[ndf['len2'] == 100]
ndf = ndf.sample(data_1.shape[0])

In [12]:
data_2 = pd.concat([pd.DataFrame(ndf['target_embd'].values.tolist(), columns=['Target_Vec_'+str(i) for i in range(0, 30)]), 
               pd.DataFrame(ndf['drug_fingerprint'].values.tolist(), columns=['Drug_Vec_'+str(i) for i in range(0, 100)])], axis=1)

data_2['Class'] = 0

In [13]:
data = pd.concat([data_1, data_2])

In [13]:
pickle.dump(data,open("dataset_vBalanced.pkl", "wb"))